<h1 align="center"> Research: Clickhouse vs Vertica </h1>

## 1. Preparing the storage cluster for operation

### `Clickhouse`

In [1]:
from clickhouse_driver import connect

clickhouse = connect(dsn='clickhouse://clickhouse:9000/default?user=default').cursor()

clickhouse.execute("""
    CREATE TABLE IF NOT EXISTS video_progress (
        user_id UUID,
        film_id UUID,
        frame UInt32,
        event_time DateTime
    ) Engine=MergeTree() ORDER BY user_id;
""")

### `Vertica`

In [2]:
from vertica_python import connect

vertica = connect(dsn='vertica://vertica:5433/docker?user=dbadmin&autocommit=true').cursor()

vertica.execute("""
    CREATE TABLE IF NOT EXISTS video_progress (
        user_id UUID,
        film_id UUID,
        frame INTEGER,
        event_time TIMESTAMP
    ) ORDER BY user_id;
""")

## 2. Script for data generation

In [3]:
from faker import Faker

fake = Faker()

def gendata():
    return {
        'user_id': fake.uuid4(),
        'film_id': fake.uuid4(),
        'frame': fake.random_int(max=4294967295),
        'event_time': fake.past_datetime(start_date='-1y'),
    }

## 3. Data insertion testing (10,000,000 rows)

In [4]:
batch_size = 10000
multiplier = 1000

### `Clickhouse`

In [5]:
%%time
for _ in range(multiplier):
    clickhouse.execute(
        'INSERT INTO video_progress (user_id, film_id, frame, event_time) VALUES',
        [gendata() for _ in range(batch_size)],
    )

CPU times: user 4min 6s, sys: 298 ms, total: 4min 7s
Wall time: 4min 9s


In [6]:
clickhouse.execute('SELECT COUNT(*) FROM video_progress')
clickhouse.fetchone()

(10000000,)

### `Vertica`

In [7]:
%%time
for _ in range(multiplier):
    vertica.executemany(
        'INSERT INTO video_progress (user_id, film_id, frame, event_time) VALUES (:user_id, :film_id, :frame, :event_time)',
        [gendata() for _ in range(batch_size)],
    )

CPU times: user 5min 30s, sys: 1.23 s, total: 5min 32s
Wall time: 6min 42s


In [8]:
vertica.execute('SELECT COUNT(*) FROM video_progress')
vertica.fetchone()

[10000000]

## 4. Data reading test (for 1 day)

In [9]:
from datetime import datetime, timedelta

yesterday = datetime.now() - timedelta(days=1)

### `Clickhouse`

In [10]:
%%time
clickhouse.execute(
    'SELECT * FROM video_progress WHERE event_time  > %(datetime)s ORDER BY event_time',
    {'datetime': yesterday},
)

CPU times: user 148 ms, sys: 5 ms, total: 153 ms
Wall time: 768 ms


In [11]:
clickhouse.fetchone()

(UUID('e88541df-6b00-476a-9791-35eaaeb1d534'),
 UUID('fa8ddbf8-853b-40b4-9e94-3f27a6326a4a'),
 1635196973,
 datetime.datetime(2023, 1, 25, 15, 4, 56))

### `Vertica`

In [12]:
%%time
vertica.execute(
    'SELECT * FROM video_progress WHERE event_time  > :datetime ORDER BY event_time',
    {'datetime': yesterday},
)

CPU times: user 2.87 ms, sys: 4 ms, total: 6.87 ms
Wall time: 1.38 s


In [13]:
vertica.fetchone()

[UUID('c0e925cd-1864-405a-b9d0-d33e8c68e1ba'),
 UUID('cdac1275-7220-4935-ba06-77827a0de199'),
 3831947341,
 datetime.datetime(2023, 1, 25, 15, 4, 57)]

## 4. Testing of data aggregation (frames)

### `Clickhouse`

In [14]:
%%time
clickhouse.execute('SELECT MAX(frame), MIN(frame), SUM(frame), AVG(frame) FROM video_progress')

CPU times: user 8.76 ms, sys: 3 ms, total: 11.8 ms
Wall time: 71.9 ms


In [15]:
clickhouse.fetchone()

(4294967002, 632, 21479922194684053, 2147992219.4684052)

### `Vertica`

In [16]:
%%time
vertica.execute('SELECT MAX(frame), MIN(frame), SUM(frame), AVG(frame) FROM video_progress')

CPU times: user 154 ms, sys: 7.03 ms, total: 161 ms
Wall time: 440 ms


In [17]:
vertica.fetchone()

[4294966375, 123, 21479771290323761, 2147977129.03238]

### **Conclusion: ClickHouse is faster at handling analytic queries than Vertica.**